# Example use of the Sonic module Nodosus.jl

See https://julialang.org, https://juliadb.org and https://juliastats.org for reference.

In [1]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

Activating environment at `~/Documents/Calendar/Work/Noda/Code/Sonic.jl/Project.toml`


In [2]:
import Dates, JuliaDB, Random, StatsPlots

In [3]:
using StatsPlots
ENV["GKSwstype"] = "100"
gr(fmt = :png)

Plots.GRBackend()

Get secret token,

In [4]:
include("../src/LinckiiSecrets.jl")
secret = LinckiiSecrets.keab

(url = "https://canary.noda.se/~keab", key = "e86d93131e6bc12ae701d036e696cf8b")

In [26]:
include("../src/Linckii.jl")
include("../src/Nodosus.jl")

Main.Nodosus

Load site,

In [6]:
@time site = Linckii.loadsite(secret, :nodes, :devices, :sensors)
()

 26.587703 seconds (24.79 M allocations: 1.665 GiB, 1.71% gc time)


()

Specify patterns,

In [7]:
p, q = Dates.Minute(15), Dates.Minute(4 * 4 * 15)
pp0 = rows -> Nodosus.preprocess0(rows, p, q)
pp1 = rows -> Nodosus.preprocess1(rows, p, q)
pp2 = rows -> Nodosus.preprocess2(rows, p, q)
patterns = Dict(
    4 => Dict(
        :meter_effect           => pp1,
        :meter_heatenergy       => pp2,
        :meter_primreturntemp   => pp1,
        :meter_primsupplytemp   => pp1,
        :meter_volume           => pp2,
        :meter_volumeflow       => pp1,
        :outdoortemp            => pp1,
        :outdoortemp_offset     => pp0,
    ),
    6 => Dict(
        :cloudiness             => pp1,
        :outdoortemp            => pp1,
        :wind_direction         => pp1,
        :wind_speed             => pp1,
    ),
)

Dict{Int64,Dict{Symbol,V} where V} with 2 entries:
  4 => Dict{Symbol,Function}(:meter_primreturntemp=>#9,:meter_effect=>#9,:outdo…
  6 => Dict(:cloudiness=>#9,:outdoortemp=>#9,:wind_direction=>#9,:wind_speed=>#…

Get pattern nodes,

In [8]:
rs = JuliaDB.rows(Linckii.get_pattern_nodes(patterns; site...))
if true
    rs = [(node_id = 62, device_id = 4), (node_id = 93, device_id = 4)] # [rs[Random.rand(1 : end)]]
end

2-element Array{NamedTuple{(:node_id, :device_id),Tuple{Int64,Int64}},1}:
 (node_id = 62, device_id = 4)
 (node_id = 93, device_id = 4)

Load data,

In [9]:
data = Dict()

Dict{Any,Any} with 0 entries

In [30]:
@time for r in rs
    filepath = "$(Linckii.datapath(secret, r.node_id))"
    try
        r_pattern = patterns[r.device_id]
        data[r.node_id] = JuliaDB.unstack(
            Nodosus.preprocess(
                Linckii.loaddata(secret, r.node_id, keys(r_pattern)),
                r_pattern,
            ),
        )
        println("$(filepath): Done")
    catch e
        println("$(filepath): $(e)")
    end
end

db/linckii/~keab/data/62: MethodError(setindex!, (Table with 48572 rows, 3 columns:
datetime             variable               value
─────────────────────────────────────────────────
2018-02-14T12:58:06  :meter_primreturntemp  41.0
2018-02-14T13:08:06  :meter_primreturntemp  40.0
2018-02-14T13:18:06  :meter_primreturntemp  44.0
2018-02-14T13:28:05  :meter_primreturntemp  43.0
2018-02-14T13:38:06  :meter_primreturntemp  43.0
2018-02-14T13:48:13  :meter_primreturntemp  36.0
2018-02-14T13:58:06  :meter_primreturntemp  44.0
2018-02-14T14:08:05  :meter_primreturntemp  42.0
2018-02-14T14:18:14  :meter_primreturntemp  43.0
2018-02-14T14:28:06  :meter_primreturntemp  36.0
2018-02-14T14:38:06  :meter_primreturntemp  43.0
2018-02-14T14:48:05  :meter_primreturntemp  43.0
⋮
2019-03-28T06:28:14  :meter_primreturntemp  40.0
2019-03-28T06:38:05  :meter_primreturntemp  36.0
2019-03-28T06:48:21  :meter_primreturntemp  40.0
2019-03-28T06:58:13  :meter_primreturntemp  34.0
2019-03-28T07:08:06  :meter_pr

Random choice,

In [31]:
r = rs[Random.rand(1 : end)]

(node_id = 62, device_id = 4)

In [32]:
data[r.node_id]

(datetime = 2018-02-14T23:08:06, variable = :meter_primreturntemp, value = 45.0)

Simple graphics,

In [33]:
@df data[r.node_id] plot(:datetime, [:meter_primreturntemp :meter_primsupplytemp :outdoortemp :outdoortemp_offset])

ErrorException: Only tables are supported

In [34]:
@df data[r.node_id] plot(:datetime, [:meter_heatenergy])

ErrorException: Only tables are supported

In [35]:
@df data[r.node_id] plot(:datetime, [:meter_volume])

ErrorException: Only tables are supported